# As per https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/
I will test both a simple RNN and CNN

https://github.com/streamride/wakeworddetection/blob/main/model.py

In [120]:

class SimpleRNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super().__init__()
        self.rnn = nn.LSTM(128, 256, batch_first=False)
        self.classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Linear(64,1)
        )
        # self.fc1 = nn.Linear(256, 128)
        # self.fc = nn.Linear(128, 1)
        self.dropout = nn.Dropout(dropout_rate)

        self.layer_norm = nn.LayerNorm(128)

    # def init_hidden(self, batch_size):
        # return 

    def forward(self, x):
        x = x.squeeze(1)
        x = self.layer_norm(x)
        # print(x.shape)
        # x = x.transpose(0,1)
        # x = x.transpose(1,2)
        
        # print(x.shape)
        x, (hidden_last, cell_last) = self.rnn(x)
        hidden_last = self.dropout(hidden_last)
        # print(x.shape)
        # print(hidden_last.shape)
        hidden_last = hidden_last.squeeze(0)
        x = self.classifier(hidden_last)
        # print(x.shape)
        return x

In [121]:
model = SimpleRNN()

x = torch.zeros(1,128,128)
model(x).shape

torch.Size([128, 1])

In [111]:
class SimpleRNN(nn.Module):
    def __init__(self, time = 128,dropout_rate=0.5):
        super().__init__()
        
        # in https://github.com/streamride/wakeworddetection/blob/main/model.py - the number of hidden layers increases then decreases
        self.time = time
        hidden   = int(self.time * 2)
        hidden2  = int(self.time / 2)
        
        
        
        self.rnn = nn.LSTM(self.time, hidden, batch_first=False)
        self.classifier = nn.Sequential(
            nn.Linear(hidden, time),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(time, hidden2),
            nn.LeakyReLU(),
            nn.Linear(hidden2,1)
        )
        # self.fc1 = nn.Linear(256, 128)
        # self.fc = nn.Linear(128, 1)
        self.dropout = nn.Dropout(dropout_rate)

        self.layer_norm = nn.LayerNorm(time)

    # def init_hidden(self, batch_size):
        # return 

    def forward(self, x):
        x = x.squeeze(1)
        x = self.layer_norm(x)
        # print(x.shape)
        # x = x.transpose(0,1)
        # x = x.transpose(1,2)
        
        # print(x.shape)
        x, y = self.rnn(x)
        #hidden_last, cell_last = y
        y[0] = self.dropout(hidden_last)
        # print(x.shape)
        # print(hidden_last.shape)
        hidden_last = hidden_last.squeeze(0)
        x = self.classifier(hidden_last)
        
        x = x.reshape(1,-1)
        # print(x.shape)
        return x




In [112]:
from torchsummary import summary
input_time = 1375

model = SimpleRNN(time = 128)

summary(model, (1,64,time))    

AttributeError: 'tuple' object has no attribute 'size'

In [80]:
freq,time = 101,1375
model = SimpleRNN(time=time)
x = torch.ones(1,freq,time)
model(x).shape

torch.Size([1, 101])

In [101]:
class SimpleCNN(nn.Module):
    def __init__(self, dropout_rate=0.5, freq = 140, time = 128):
        super().__init__()
        
        self.dropout = nn.Dropout(dropout_rate)
        self.layer_norm = nn.LayerNorm(time)

        self.cnn_block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=time, kernel_size=3),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

        self.cnn_block2 = nn.Sequential(
            nn.Conv2d(in_channels=freq, out_channels=64, kernel_size=3),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

        self.cnn_block3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.fc1 = nn.Linear(32*14*14, 128)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    
    def forward(self, x):
        # print(x.shape)
        batch_size, channels, _, _ = x.shape
        x = self.layer_norm(x)
        print(x.shape)
        x = self.cnn_block1(x)
        
        print(x.shape)
        x = self.cnn_block2(x)
        print(x.shape)
        x = self.cnn_block3(x)
        print(x.shape)
        
        x = x.view(batch_size, -1)
        x = F.relu(self.fc1(x))
        print(x.shape)
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        print(x.shape)
        x = self.output(x)
        return x

In [102]:

a = 4480
b = 6272

b/freq

49.0

In [103]:
freq,time = 128,128

model2 = SimpleCNN()
x = torch.ones(1,1,freq,time)
model2(x).shape

torch.Size([1, 1, 128, 128])
torch.Size([1, 128, 63, 63])


RuntimeError: Given groups=1, weight of size [64, 140, 3, 3], expected input[1, 128, 63, 63] to have 140 channels, but got 128 channels instead

In [54]:
rnn = nn.LSTM(128, 256, batch_first=True)

x1 = torch.ones(100,128)
x, (hidden_last, cell_last) = rnn(x1)

print(x.shape,hidden_last.shape,cell_last.shape)


seq =  nn.Sequential(
            nn.Conv1d(in_channels=100, out_channels=256, kernel_size=3),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Dropout( 0.8)
        )

x2 = seq(x.reshape(1,-1))

x2.shape


torch.Size([100, 256]) torch.Size([1, 256]) torch.Size([1, 256])


RuntimeError: Given groups=1, weight of size [256, 100, 3], expected input[1, 1, 25600] to have 100 channels, but got 1 channels instead

In [43]:
model = SimpleRNN()
x = torch.ones(100,128)
model(x).shape

torch.Size([100, 128])
torch.Size([100, 128])
torch.Size([1, 1])


torch.Size([1, 1])

In [35]:
model2 = SimpleCNN()
x = torch.ones(1,100,1)
model2(x).shape

ValueError: not enough values to unpack (expected 4, got 3)

In [21]:
nn.LayerNorm(128)(x).shape

torch.Size([1, 100, 128])

In [26]:
summary(model, (100,128))    

AttributeError: 'tuple' object has no attribute 'size'

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class SimpleRNN(nn.Module):
    def __init__(self, spec_time, dropout_rate=0.5):
        super().__init__()
        hidden_layer = int(spec_time/2)
        hidden_layer2 = int(hidden_layer/2)
        
        self.rnn = nn.LSTM(spec_time,hidden_layer, batch_first=True)
        self.classifier = nn.Sequential(
            nn.Linear(spec_time,hidden_layer),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_layer, hidden_layer2),
            nn.LeakyReLU(),
            nn.Linear(hidden_layer2,1)
        )
        # self.fc1 = nn.Linear(256, 128)
        # self.fc = nn.Linear(128, 1)
        self.dropout = nn.Dropout(dropout_rate)

        self.layer_norm = nn.LayerNorm(hidden_layer2)

    # def init_hidden(self, batch_size):
        # return 

    def forward(self, x):
        x = x.squeeze(1)
        x = self.layer_norm(x)
        # print(x.shape)
        # x = x.transpose(0,1)
        # x = x.transpose(1,2)
        
        # print(x.shape)
        x, (hidden_last, cell_last) = self.rnn(x)
        hidden_last = self.dropout(hidden_last)
        # print(x.shape)
        # print(hidden_last.shape)
        hidden_last = hidden_last.squeeze(0)
        x = self.classifier(hidden_last)
        # print(x.shape)
        return x

In [9]:
spec_time = 128

hidden_layer = int(2*spec_time)
hidden_layer2 = int(spec_time/2)

nn.Linear(spec_time,hidden_layer)(torch.zeros(hidden_layer,spec_time)).shape

torch.Size([256, 256])

In [13]:
from torchsummary import summary
input_time = 1375

model = SimpleRNN(spec_time = 128)

print(f"input shape: (1,{spec_time},{input_time})")  
summary(model, (1,64,128))    

input shape: (1,128,1375)


RuntimeError: Given normalized_shape=[32], expected input with shape [*, 32], but got input of size[2, 64, 128]

In [20]:
spec_time = 128
hidden_layer = int(2*spec_time)
hidden_layer2 = int(spec_time/2)
dropout_rate = 0.8

rnn = nn.LSTM(spec_time,hidden_layer, batch_first=True)
classifier = nn.Sequential(
nn.Linear(hidden_layer, spec_time),
nn.LeakyReLU(),
nn.Dropout(dropout_rate),
nn.Linear(spec_time, hidden_layer2),
nn.LeakyReLU(),
nn.Linear(hidden_layer2,1)
)
# self.fc1 = nn.Linear(256, 128)
# self.fc = nn.Linear(128, 1)
dropout = nn.Dropout(dropout_rate)

layer_norm = nn.LayerNorm(hidden_layer2)